**Import Libraries**

In [ ]:
# !pip install python-telegram-bot

In [ ]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import numpy as np
import urllib
from urllib import request
import os
import cv2
from tensorflow.keras.models import load_model
import config as config
%run utils.ipynb
import warnings
warnings.filterwarnings('ignore')

**Connect to Drive**

Uncomment if using Google Colab

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')
# os.chdir(config.PATH)

**Create Telegram Bot**

In [ ]:
token = config.TOKEN

In [ ]:
updater = Updater(token=token, use_context=True)

In [ ]:
dispatcher = updater.dispatcher

In [ ]:
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text="I'm a bot, please send me your pic!")

In [ ]:
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [ ]:
def photo(update, context):
    vgg_prediction = []
    inception_prediction = []
    xception_prediction = []
    user = update.message.from_user
    photo_file = update.message.photo[-1].get_file().file_path
    # photo_file.download('user_photo.jpg') 
    img = image_url_to_numpy_array_urllib(photo_file)
    if img == 'rectangular':
        message = "What a great pic, too bad that it is too rectangular for me to process, let's try squarer pic :)"
        context.bot.send_message(chat_id = update.effective_chat.id, text=message)
        return 0 
    img = img.reshape(1,224,224,3)
    vgg_model = load_model('vgg_model.h5')
    inception_model = load_model('inception_model.h5')
    xception_model = load_model('xception_model.h5')
    y_pred_vgg = vgg_model.predict(img)
    y_pred_inception = inception_model.predict(img)
    y_pred_xception = xception_model.predict(img)
    col_dict = {0: '#animal',
                1: '#human',
                2: '#mountain',
                3: '#nature',
                4: '#outdoors',
                5: '#person',
                6: '#plant',
                7: '#sea',
                8: '#tree',
                9: '#water'}
    
    for i in range(10):
        if y_pred_vgg[0][i] > 0.5:
            vgg_prediction.append(col_dict[i])
        if y_pred_inception[0][i] > 0:
            inception_prediction.append(col_dict[i])
        if y_pred_xception[0][i] > 0:
            xception_prediction.append(col_dict[i])
    
    if len(y_pred_vgg) > 0:
        vgg_message = 'VGG16: ' + ' '.join(vgg_prediction)
    else:
        vgg_message = "VGG16 can't recognize the pic, let's try one more..."

    if len(y_pred_inception) > 0:
        inception_message = 'INCEPTION: ' + ' '.join(inception_prediction)
    else:
        inception_message = "INCEPTION can't recognize the pic, let's try one more..."

    if len(y_pred_xception) > 0:
        xception_message = 'XCEPTION: ' + ' '.join(xception_prediction)
    else:
        xception_message = "XCEPTION can't recognize the pic, let's try one more..."

    message = vgg_message + '\n' + inception_message + '\n' + xception_message

    context.bot.send_message(chat_id = update.effective_chat.id, 
                             text = message)

In [ ]:
echo_handler = MessageHandler(Filters.photo & (~Filters.command), photo)
dispatcher.add_handler(echo_handler)

In [ ]:
updater.start_polling()

In [ ]:
# updater.stop()